# Explore here

In [24]:
import pandas as pd
from utils import db_connect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
from sklearn.metrics import *
from sklearn.ensemble import BaggingClassifier

In [2]:
engine = db_connect()

# Consultar la tabla en la base de datos y crear un DataFrame
total_data = pd.read_sql_table('tabla playstore', engine)

pd.set_option('display.max_columns', None)  # muestra todas las columnas del dataframe
total_data.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [3]:
total_data.to_csv('/workspaces/Naive-Bayes/data/raw/playstore.csv', index=False)

tenemos solo 3 variables: 2 predictoras y una etiqueta dicotómica. De las dos predictoras, realmente solo nos interesa la parte del comentario, ya que el hecho de clasificar un comentario en positivo o negativo dependerá de su contenido, no de la aplicación de la que se haya escrito. Por lo tanto, la variable package_name habría que eliminarla.

In [4]:
total_data.drop(['package_name'], axis=1, inplace=True)

In [5]:
total_data.shape

(891, 2)

In [6]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   review    891 non-null    object
 1   polarity  891 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.0+ KB


In [7]:
total_data["review"] = total_data["review"].str.strip().str.lower()
total_data.head()

,review,polarity
0,privacy at least put some option appear offlin...,0
1,"messenger issues ever since the last update, i...",0
2,profile any time my wife or anybody has more t...,0
3,the new features suck for those of us who don'...,0
4,forced reload on uploading pic on replying com...,0


In [8]:
X = total_data['review']
y = total_data['polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
vectorizer = CountVectorizer(stop_words = "english")  # stop_words = "english" eliminará las palabras comunes del inglés del texto antes de generar la matriz de recuentos

X_train_v = vectorizer.fit_transform(X_train).toarray()      
X_test_v = vectorizer.transform(X_test).toarray()

X_train = vectorizer.fit_transform(X_train).toarray(): Convierte los documentos de texto en el conjunto de datos de entrenamiento (X_train) en una matriz de recuentos de tokens. fit_transform() primero ajusta el vectorizador al conjunto de datos de entrenamiento (X_train) y luego lo transforma en una matriz de recuentos.  

X_test = vectorizer.transform(X_test).toarray(): Transforma los documentos de texto en el conjunto de datos de prueba (X_test) en una matriz de recuentos de tokens utilizando el mismo vectorizador que se ajustó al conjunto de datos de entrenamiento. transform() simplemente aplica la transformación aprendida previamente en el conjunto de datos de entrenamiento al conjunto de datos de prueba.

 - Matriz dispersa: Solo se almacenan los valores distintos de cero junto con sus ubicaciones, lo que ahorra memoria. Esto es útil cuando se trabaja con matrices muy grandes donde la mayoría de los elementos son cero. Algunos formatos de matriz dispersa comunes son el formato de lista de listas (LIL), el formato de matriz comprimida en fila (CSR) y el formato de matriz comprimida en columna (CSC).

 - Matriz densa: Todos los elementos de la matriz se almacenan en la memoria, independientemente de si son cero o no. Esto puede llevar a un uso significativo de memoria, especialmente para matrices grandes con muchos ceros. Sin embargo, las operaciones en matrices densas suelen ser más rápidas que en matrices dispersas, especialmente en operaciones de álgebra lineal.

La elección entre usar una matriz dispersa o densa depende del contexto y de las características de los datos:

Para conjuntos de datos grandes con muchos ceros, como matrices de documentos término-documento en procesamiento de lenguaje natural, las matrices dispersas son más eficientes en términos de memoria.
Para operaciones de álgebra lineal y cálculos intensivos, como la multiplicación de matrices, las matrices densas suelen ser más eficientes computacionalmente.
Algunos algoritmos de aprendizaje automático pueden requerir entradas en forma de matrices densas, mientras que otros pueden admitir matrices dispersas. Por lo tanto, 

Modelo Naive Bayes

MultinomialNB:

 - Características:
Se utiliza cuando las características son representativas de conteos o frecuencias de eventos, como la frecuencia de palabras en un documento.
Asume que las características se distribuyen multinomialmente, lo que significa que cada característica representa el número de veces que ocurre un evento en particular dentro de una muestra.
 - Uso:
Es ampliamente utilizado en clasificación de texto y minería de texto, donde las características representan frecuencias de palabras o términos en un documento.

In [10]:
model = MultinomialNB()
model.fit(X_train_v, y_train)

MultinomialNB()

In [11]:
y_pred = model.predict(X_test_v)
y_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0])

In [12]:
train_pred = model.predict(X_train_v)
test_pred = model.predict(X_test_v)

In [13]:
print(accuracy_score(y_train, train_pred))
print(accuracy_score(y_test, test_pred))

0.952247191011236
0.8491620111731844


In [14]:
# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89       118
           1       0.85      0.67      0.75        61

    accuracy                           0.85       179
   macro avg       0.85      0.81      0.82       179
weighted avg       0.85      0.85      0.84       179



BernoulliNB:

 - Características:
Se utiliza cuando las características son binarias, es decir, cada característica representa la presencia o ausencia de un evento.
Asume que las características se distribuyen de acuerdo con una distribución de Bernoulli.
 - Uso:
Es útil para problemas de clasificación donde las características son binarias, como la clasificación de documentos por temas (si un término aparece o no en un documento), la detección de spam en correos electrónicos (presencia o ausencia de ciertas palabras clave) y la clasificación de imágenes (presencia o ausencia de ciertas características visuales).

In [15]:
model_bernoulli = BernoulliNB().fit(X_train_v, y_train)

In [16]:
y_pred_b = model_bernoulli.predict(X_test_v)
y_pred_b

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0])

In [17]:
print(classification_report(y_test, y_pred_b))

              precision    recall  f1-score   support

           0       0.75      0.96      0.84       118
           1       0.83      0.39      0.53        61

    accuracy                           0.77       179
   macro avg       0.79      0.68      0.69       179
weighted avg       0.78      0.77      0.74       179



GaussianNB:

 - Características:
Se utiliza cuando las características son continuas y se asume que siguen una distribución gaussiana (normal).
Es útil cuando se trabaja con características numéricas que pueden aproximarse a una distribución normal.
 - Uso:
Se utiliza comúnmente en aplicaciones donde las características se distribuyen aproximadamente de acuerdo con una distribución gaussiana, como el análisis de datos biométricos, el reconocimiento facial y la clasificación de textos con características continuas como la frecuencia de palabras.

In [18]:
model_gaussian = GaussianNB().fit(X_train_v, y_train)

In [19]:
y_pred_g = model_gaussian.predict(X_test_v)
y_pred_g

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0])

In [20]:
print(classification_report(y_test, y_pred_g))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       118
           1       0.70      0.62      0.66        61

    accuracy                           0.78       179
   macro avg       0.76      0.74      0.75       179
weighted avg       0.78      0.78      0.78       179



El mejor modelo es el inicial, el multinomial con una precisión del 85%, frente al de Benoulli y el gausiano, con una precisión del 77% y 78% respectivamente.

Optimización del Modelo Multinomial

In [21]:
hyperparams = {
    "alpha": np.linspace(0.1, 5.0, 100),  # Parámetro de suavizado de Laplace
    "fit_prior": [True, False]       # Si se deben aprender las probabilidades a priori de las clases
}

# Instancia de GridSearchCV con el modelo, hiperparámetros y número de folds
grid = GridSearchCV(model, hyperparams, scoring="accuracy", cv=15)

In [22]:
grid.fit(X_train_v, y_train)

GridSearchCV(cv=15, estimator=MultinomialNB(),
             param_grid={'alpha': array([0.1       , 0.14949495, 0.1989899 , 0.24848485, 0.2979798 ,
       0.34747475, 0.3969697 , 0.44646465, 0.4959596 , 0.54545455,
       0.59494949, 0.64444444, 0.69393939, 0.74343434, 0.79292929,
       0.84242424, 0.89191919, 0.94141414, 0.99090909, 1.04040404,
       1.08989899, 1.13939394, 1.18888889, 1.23838384, 1.28787879,
       1.33737374, 1.3...
       3.56464646, 3.61414141, 3.66363636, 3.71313131, 3.76262626,
       3.81212121, 3.86161616, 3.91111111, 3.96060606, 4.01010101,
       4.05959596, 4.10909091, 4.15858586, 4.20808081, 4.25757576,
       4.30707071, 4.35656566, 4.40606061, 4.45555556, 4.50505051,
       4.55454545, 4.6040404 , 4.65353535, 4.7030303 , 4.75252525,
       4.8020202 , 4.85151515, 4.9010101 , 4.95050505, 5.        ]),
                         'fit_prior': [True, False]},
             scoring='accuracy')

In [23]:
# Muestra los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid.best_params_)

# Muestra la precisión del mejor modelo
print("Precisión del mejor modelo:", grid.best_score_)

Mejores hiperparámetros: {'alpha': 3.9606060606060614, 'fit_prior': False}
Precisión del mejor modelo: 0.8062352245862885


Después de intentar diferentes enfoques de optimización (GridSearch y RandomizedSearch) y ajuste, el modelo sigue sin mejorar su rendimiento.

- Búsqueda con Bayes y validación cruzada

In [37]:
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
from skopt.space import Real, Categorical, Integer

In [38]:
search_space = {
    'alpha': Real(0.01, 10.0, prior='log-uniform')  # Distribución de probabilidad para alpha
}

In [39]:
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [41]:
# Inicializa BayesSearchCV
bayes_search = BayesSearchCV(
    model, 
    search_space, 
    scoring='accuracy', 
    cv=cv_strategy, 
    n_iter=50,  # Número de iteraciones para la búsqueda
    random_state=42
)

bayes_search.fit(X_train_v, y_train)

BayesSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
              estimator=MultinomialNB(), random_state=42, scoring='accuracy',
              search_spaces={'alpha': Real(low=0.01, high=10.0, prior='log-uniform', transform='normalize')})

In [42]:
print("Mejores hiperparámetros:", bayes_search.best_params_)
print("Precisión del mejor modelo:", bayes_search.best_score_)

Mejores hiperparámetros: OrderedDict([('alpha', 1.4875506095091737)])
Precisión del mejor modelo: 0.8160543681670441


 - Optimización Bagging
 
 (El bagging es una técnica de ensamblaje que combina múltiples modelos de manera paralela, entrenándolos en diferentes subconjuntos aleatorios del conjunto de datos y promediando sus predicciones para mejorar la precisión y reducir el sobreajuste.)

In [28]:
bagging_classifier = BaggingClassifier(
    estimator=model,
    n_estimators=10,  # Número de modelos en el ensamblaje (puedes ajustar según sea necesario)
    random_state=42
    )

In [29]:
bagging_classifier.fit(X_train_v, y_train)

BaggingClassifier(estimator=MultinomialNB(), random_state=42)

In [30]:
train_pred_b = bagging_classifier.predict(X_train_v)
test_pred_b = bagging_classifier.predict(X_test_v)

In [31]:
print(accuracy_score(y_train, train_pred_b))
print(accuracy_score(y_test, test_pred_b))

0.9367977528089888
0.8491620111731844


BaggingClassifier proporciona una precisión similar en el conjunto de prueba pero ligeramente peor en el conjunto de entrenamiento en comparación con el modelo original, puede deberse a varias razones:

 - Varianza reducida: El bagging (Bootstrap Aggregating) es una técnica que combina múltiples modelos de aprendizaje para mejorar la generalización y reducir la varianza. Aunque cada modelo individual en el ensemble (ensamblaje) puede ser menos preciso en el conjunto de entrenamiento que el modelo original, la combinación de estos modelos puede proporcionar una mejor generalización y, por lo tanto, una precisión similar o incluso mejor en el conjunto de prueba.

 - Sesgo aumentado: Sin embargo, al combinar múltiples modelos, el sesgo del ensemble puede aumentar ligeramente en comparación con el modelo original. Esto significa que el ensemble puede no ajustarse tan bien a los datos de entrenamiento como lo hace el modelo original, lo que resulta en una ligera reducción de la precisión en el conjunto de entrenamiento.

 - Mayor robustez: El ensemble puede ser más robusto y menos propenso al sobreajuste que el modelo original. Esto puede deberse a que el bagging reduce la dependencia de un solo modelo y su sensibilidad a los datos de entrenamiento específicos.

En resumen, una precisión ligeramente peor en el conjunto de entrenamiento pero similar en el conjunto de prueba sugiere que el BaggingClassifier está proporcionando una buena generalización y puede ser preferible en términos de robustez y capacidad de generalización en comparación con el modelo original.